<a href="https://colab.research.google.com/github/JennaBeutler/APRD-6342/blob/master/Exercise_3_LASSO_regression_for_fat_sales_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Exercise 3: LASSO regression for fat sales data
### APRD 6342  
### Fall 2019  
### Jenna Beutler

Importing all necessary packages and mounting the drive  
uploading data into Colab

In [0]:
import pandas as pd
import pandas
from sklearn.linear_model import LassoLarsCV
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 

from google.colab import drive
drive.mount('/content/drive')

In [0]:
path = '/content/drive/My Drive/Colab Notebooks/Python Exercises/finalmaster-ratios.csv'
alldata = pd.read_csv(path)
alldata.head()

,# Purchases,B01001001,B01001002,B01001003,B01001004,B01001005,B01001006,B01001007,B01001008,B01001009,B01001010,B01001011,B01001012,B01001013,B01001014,B01001015,B01001016,B01001017,B01001018,B01001019,B01001020,B01001021,B01001022,B01001023,B01001024,B01001025,B01001026,B01001027,B01001028,B01001029,B01001030,B01001031,B01001032,B01001033,B01001034,B01001035,B01001036,B01001037,B01001038,B01001039,...,B15002013,B15002014,B15002015,B15002016,B15002017,B15002018,B15002019,B15002020,B15002021,B15002022,B15002023,B15002024,B15002025,B15002026,B15002027,B15002028,B15002029,B15002030,B15002031,B15002032,B15002033,B15002034,B15002035,B19001001,B19001002,B19001003,B19001004,B19001005,B19001006,B19001007,B19001008,B19001009,B19001010,B19001011,B19001012,B19001013,B19001014,B19001015,B19001016,B19001017
0,22,206252,469.226965,31.432422,35.219052,33.628765,20.121017,12.610787,6.734480,6.225394,19.432539,28.101546,28.421543,26.390047,31.989993,31.359696,32.116052,32.213021,12.184124,18.361034,9.454454,15.175610,16.281054,11.025348,6.230243,4.518744,530.773035,31.999690,34.322091,32.649380,20.101623,12.513818,8.072649,6.021760,22.923414,31.335454,31.558482,31.063941,36.082074,34.845723,...,64.610300,31.449746,58.735313,20.071053,6.726751,5.882267,543.803963,6.974272,2.504332,5.904107,11.917415,10.767170,18.141844,19.779852,10.956451,181.418442,26.717724,85.271036,54.243532,72.647457,30.816383,2.831933,2.912014,1000,105.667996,82.298375,68.141163,67.336195,63.566902,59.439845,49.409690,53.306757,42.318307,83.167229,89.249208,102.141470,52.872330,36.440765,23.446284,21.197485
1,7,61399,486.538869,22.899396,21.531295,27.036271,16.808091,28.355511,18.192479,13.534422,21.466148,24.886399,23.534585,21.319565,27.101419,30.961416,37.117868,36.466392,12.557208,20.554081,12.182609,15.651721,20.668089,15.961172,10.423623,7.329110,513.461131,18.974250,23.404290,23.892897,17.036108,35.310021,18.534504,17.101256,22.785387,22.150198,22.622518,21.303279,26.971123,32.329517,...,56.929829,46.381727,65.707446,35.509451,16.782205,9.201536,515.086529,3.017306,1.047329,1.371503,6.358785,4.937410,8.303825,9.700264,7.555733,174.155902,25.834123,60.146626,62.440776,76.604658,55.383771,8.977108,9.251409,1000,71.289558,59.062447,54.704688,60.966323,53.012354,60.881706,59.231680,50.093078,40.700626,92.612963,117.363344,113.344051,75.774243,33.000508,33.169741,24.792689
2,3,73170,489.859232,28.905289,36.271696,28.235616,21.566216,12.218122,7.243406,7.380074,16.933169,24.914582,26.896269,31.802651,30.531639,36.258029,35.998360,33.429001,13.625803,19.406861,12.245456,14.664480,21.169878,15.293153,8.610086,6.259396,510.140768,26.171928,30.681973,31.925653,19.789531,10.072434,5.056717,6.218396,15.757824,24.449911,26.595599,27.210605,37.556376,37.050704,...,54.602613,40.613027,43.363788,12.280185,5.796247,3.438452,523.980745,5.422930,4.224384,11.828274,18.331860,15.089891,21.731015,18.685529,7.014441,155.241183,45.466156,71.185775,65.802142,56.272718,24.580018,1.689753,1.414677,1000,102.538696,82.960331,74.828305,79.133495,66.081252,78.245122,63.996993,47.322923,42.505211,70.420610,90.033143,98.677692,54.703249,20.125056,11.890525,16.537397
3,94,251724,505.585483,32.054949,31.757004,28.102207,18.651380,12.080692,7.035483,7.686991,25.790151,42.129475,35.824951,32.058922,27.677138,33.842621,38.176733,32.722347,12.493842,16.394940,11.504664,15.914255,16.394940,13.196994,8.648361,5.446441,494.414517,33.123580,28.082344,30.171934,16.863708,9.280005,5.390825,5.609318,19.453846,35.614403,32.082757,28.809331,27.911522,32.690566,...,88.227492,44.076261,87.939148,44.404973,9.671057,7.283569,502.912274,4.509700,0.980370,3.552398,5.986021,7.398907,9.740260,10.605292,7.485410,141.242417,43.078591,84.479020,52.069156,89.836451,33.932320,4.129086,3.886877,1000,61.632139,46.526521,48.437595,54.221644,51.680322,60.066684,54.790900,48.681562,43.873381,84.717507,112.204444,127.137252,83.019904,43.731067,38.851729,40.427349
4,0,37382,495.586111,25.413301,29.318924,26.162324,19.260607,12.893

For the sake of this exercise, we know the first 8 variables aren't relevant to the regression, and we want to predict the Purchases column, so we will remove them from the list of variables in our regression

In [0]:
allvariablenames = list(alldata.columns.values)
allvariablenames.remove('# Purchases')
allvariablenames.remove('B01001001')
allvariablenames.remove('B01001002')
allvariablenames.remove('B01001003')
allvariablenames.remove('B01001004')
allvariablenames.remove('B01001005')
allvariablenames.remove('B01001006')
allvariablenames.remove('B01001007')


Setting up the predictors, the targets, and the train, test split then running and fitting our LASSO model

In [0]:
predictors = alldata[allvariablenames]  
target = alldata['# Purchases']

In [0]:
pred_train, pred_test, tar_train, tar_test = train_test_split(predictors, target, test_size=.3, random_state=123) 

In [0]:
model = LassoLarsCV(cv=10, precompute=False)
model.fit(predictors, target)

In [0]:
# turning the model predictions/ coefficients into a dataframe, which is easier to work with 
predictors_model = pd.DataFrame(predictors.columns) 

# naming the first column 'label' which is the name of each predictor variable
predictors_model.columns = ['label']

# creating the second column of coefficients and pulling the coefficients from the model to fill in the dataframe
predictors_model['coeff'] = model.coef_

#prints the new dataframe
predictors_model

,label,coeff
0,B01001008,0.000000
1,B01001009,0.000000
2,B01001010,0.000000
3,B01001011,0.000000
4,B01001012,0.000000
...,...,...
177,B19001013,-0.002185
178,B19001014,0.000000
179,B19001015,0.000000
180,B19001016,0.000000


Looking at what coefficients from the LASSO model are greater than 0

In [0]:
# loops through each row of the above dataframe
for index, row in predictors_model.iterrows():
    if row['coeff'] > 0: # if the coefficient is greater than 0, print the row.
        print(row.values)
# we want to look at all coefficients greater than 0 since they are all significant under a LASSO model. 

['B01001014' 0.5330720656233966]
['B01001036' 1.6591227573088478]
['B01001037' 2.4482060101637564]
['B01001038' 2.21381020716577]
['B02001005' 0.9859155344708008]
['B02001007' 0.006591090821753925]
['B13014016' 0.0061795097515955225]
['B13014026' 0.574990933842321]
['B13014027' 1.2849909864550721]
['B13016008' 0.31692716289106393]
['B15002027' 3.9418447021477347]
['B19001017' 1.8860806124452758]


**QUESTION #1: In your own words, explain what the above lines of code are doing. Why am I doing it? Explain each line.**   
(commented in code)


**Now we know which variables most predicted sales. Because of the aforementioned feature section feature of LASSO, any coefficients that are non-zero are significant, and thus variables that truly predict unique amounts of sales. The higher the value of the coefficient, the steeper the relationship between sales and that variable.**

**Let's figure out what these Census variables actually mean. To do this, we're going to perform a simple hack, and have you Google the variable name + "Census", here's an example. Click on the first result from socialexplorer.com. For this example, I'd report: Females aged 30 to 34 Years.**  
B01001014: Males 40 - 44  
B01001036: females 30 - 34  
B01001037: females 35 - 39  
B01001038: females 40 - 44   
B02001005: Asian  
B02001007: Some other Race  
B13014016: now married  
B13014026: women 15-50 that had a baby in the past year   
B13014027: women 15-50, unmarried, no baby in past year with graduate degree  
B13016008: women 40-44 that had a baby in past year  
B15002027: 25+ females 12th grade no diploma  
B19001017: Households that make 200,000+


**QUESTION #2: Interpret each variable intuitively. What Census variables most predict sales? What does that mean, practically? Here's what I'd put for the example above. "In areas where there are more females aged 30-34, we sell more Bobo Bars."**  
Men and women roughly between the ages of 30 and 44 are more likely to buy bobo bars (fairly strong).  
Non-white, especially those that fall into the Asian and Other categories of race are slightly more likely to buy bobo bars (very small relationship).  
Now married women are also slightly more likely to buy bobo bars (very slight relationship).  
Areas with Women that had a baby in the past year will sell more bars (medium strength relationship). 
Unmarried women with graduate degrees are more likely to buy bars(medium strength relationship).  
Areas with women 25+ and no diploma will likely sell more bars(VERY strong).  
wealthier areas will probably sell more bars as well (strong).  

**QUESTION #3: If I had to report only two census variables to my boss that most steeply predicted sales, what would those be?**  
The 25+ women with no high school diploma (3.9 coef)  
females 35 - 39 (2.45 coef) 



Next, lets take a look at the mean squared error for the training and training set:

In [0]:
import sklearn as sk
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LassoCV, LarsCV, LassoLarsCV
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error, accuracy_score

In [0]:
train_error = mean_squared_error(tar_train, model.predict(pred_train))
print ('training data MSE')
print(train_error)

training data MSE
21060.761034858362


In [0]:
train_error = mean_squared_error(tar_test, model.predict(pred_test))
print ('test data MSE')
print(train_error)

test data MSE
36561.48369809522


**QUESTION #4: Are the training and text set mean squared errors similar? What does that mean practically?**  
The standard errors are both quite large, meaning that the model isn't super accurate in predicting sales of bobo bars to begin with. The goal of machine learning / regressions is to minimize the mean squared error, and this test did not. Practically, this means that this particular model isn't exactly the best for predicting who buys bobo bars. 

In [0]:
rsquared_train=model.score(pred_train,tar_train)
print ('training data R-square')
print(rsquared_train)

training data R-square
0.2733096196457573


In [0]:
rsquared_test=model.score(pred_test,tar_test)
print ('test data R-square')
print(rsquared_test)

test data R-square
0.27480133108928484


**QUESTION #5: If your boss asked, "How well does Census data, overall, predict sales?" What would you say? Why?**  
I would say that census data doesn't really predict sales because of the high mean squared errors and fairly low R-square scores. The R-squares mean that we can only predict/explain about 27 percent of the data. 

In [0]:
print("y interecept:")
print(model.intercept_)

y interecept:
-3.9606645084610435


**QUESTION #6: What is our baseline sales number? What does that mean, practically? Think back to what y-intercepts mean in regression models.**  
The y intercept is -3.96 which means that if all other variables equaled 0 (which isn't practically possible) that sales would be in the negatives. However, this just helps us scale the number of sales that actually happen given the variables that we use. 